In [2]:
# import packages
import re
import string
import pandas as pd
import numpy as np
import datetime
import time
from datetime import *

# TABLE: Pittsburgh Sleep Quality Index (PSQI)

In [ ]:
# arguments
path = r'/Users/xuqiu/desktop/8901/score_calc/'
infile = r'PSQI_DiabetesGINutritionA_DATA_2021-09-09_1326.csv'
outfile = r'pittsburgh_sleep_quality_index'

# read data from csv
samples = pd.read_csv(path+infile, dtype=str)

In [177]:
# clean empty data
samples = samples.replace(np.nan, '', regex=True)

In [178]:
#extract column values needed for score calculation 
def make_list(column_name):
    empty_ls=[]
    for item in samples[column_name]:
        if item=='':
            a=item
        else :
            a=int(item)
        empty_ls.append(a)
    return empty_ls
#score 1
# samples["quality_overall_psqi"]
# quality_list=[]
# for item in samples["quality_overall_psqi"]:
#     quality_list.append(item)
quality_list=[]
quality_list=make_list("quality_overall_psqi")
#score  2   
in30_list=[]
in30_list=make_list('sleep_in_30_mins_psqi')
#score  4
#usual_bed_time_psqi,usual_get_up_psqi=[],[]
#usual_bed_time_psqi_list=make_list('usual_bed_time_psqi')
#usual_get_up_psqi_list=make_list('usual_get_up_psqi')
    
#score  3 
hours_list=[]
hours_list=make_list('hours_of_sleep_psqi')

#score 5: 
wake_in_night_psqi_ls,use_bathroom_psqi_ls,breathe_comf_psqi_ls,cough_snore_psqi_ls,cold_psqi_ls,hot_psqi_ls,dreams_psqi_ls,pain_psqi_ls=[],[],[],[],[],[],[],[]
wake_in_night_psqi_ls=make_list('wake_in_night_psqi')
use_bathroom_psqi_ls=make_list('use_bathroom_psqi')
breathe_comf_psqi_ls=make_list('breathe_comf_psqi')
cough_snore_psqi_ls=make_list('cough_snore_psqi')
cold_psqi_ls=make_list('cold_psqi')
hot_psqi_ls=make_list('hot_psqi')
dreams_psqi_ls=make_list('dreams_psqi')
pain_psqi_ls=make_list('pain_psqi')

#score 6: sleep_meds_psqi
sleep_meds_psqi_ls=[]
sleep_meds_psqi_ls=make_list('sleep_meds_psqi')

#score 7:
stay_awake_psqi_ls, enthusiasm_psqi_ls=[],[]
stay_awake_psqi_ls=make_list('stay_awake_psqi')
enthusiasm_psqi_ls=make_list('enthusiasm_psqi')

#### Columns Renaming

In [179]:
# rename columns
samples.rename(columns={
    'cfbr':'patient_id',
    'pittsburgh_sleep_quality_index_psqi_timestamp':'timestamp_psqi',
    'loaction_psqi':'location_psqi',
    'other_restlessness':'other_restlessness_psqi',
    'pittsburgh_sleep_quality_index_psqi_complete':'completion_psqi',
    },inplace=True)

# show cleaned column names
samples.columns

Index(['record_id', 'redcap_survey_identifier', 'patient_id', 'timestamp_psqi',
       'survey_date_psqi', 'location_psqi', 'usual_bed_time_psqi',
       'number_of_mins_psqi', 'usual_get_up_psqi', 'hours_of_sleep_psqi',
       'sleep_in_30_mins_psqi', 'wake_in_night_psqi', 'use_bathroom_psqi',
       'breathe_comf_psqi', 'cough_snore_psqi', 'cold_psqi', 'hot_psqi',
       'dreams_psqi', 'pain_psqi', 'other_reasons_psqi',
       'other_how_often_psqi', 'quality_overall_psqi', 'sleep_meds_psqi',
       'stay_awake_psqi', 'enthusiasm_psqi', 'partner_psqi', 'snoring_psqi',
       'pause_breath_psqi', 'twitching_psqi', 'confusion_psqi',
       'other_restlessness_psqi', 'completion_psqi'],
      dtype='object')

#### Content Replacement

In [180]:
# change CFBR to CFBR-EM/AU-XXXX
# samples['record_id'] = samples['record_id'].replace('CFBR-', 'CFBR-EM-0', regex = True)

# dictionary for samples
sample_complete = {
    '0': 'Incomplete',
    '1': 'Unverified',
    '2': 'Complete',
}
sample_location = {
    '1':'Emory University',
    '2':'Augusta University',
}
sample_frequency = {
    '0':'Not during the past month',
    '1':'Less than once a week',
    '2':'Once or twice a week',
    '3':'Three or more times a week',
}
sample_selfRating = {
    '0':'Very good',
    '1':'Fairly good',
    '2':'Fairly bad',
    '3':'Very bad',
}
sample_difficultyLevel = {
    '0':'Not a problem at all',
    '1':'Only a very slight problem',
    '2':'Somewhat of a problem',
    '3':'A very big problem',
}
sample_bedPartner = {
    '0':'No bed partner or roommate',
    '1':'Partner/Roommate in other room',
    '2':'Partner in the same room, but not same bed',
    '3':'Partner in same bed',
}

# change option number data into text
samples = samples.replace({
    'completion_psqi': sample_complete,
    'location_psqi':sample_location,   
    'quality_overall_psqi':sample_selfRating,
    'enthusiasm_psqi':sample_difficultyLevel,
    'partner_psqi':sample_bedPartner,
    'sleep_in_30_mins_psqi':sample_frequency,
    'wake_in_night_psqi':sample_frequency,
    'use_bathroom_psqi':sample_frequency,
    'breathe_comf_psqi':sample_frequency,
    'cough_snore_psqi':sample_frequency,
    'cold_psqi':sample_frequency,
    'hot_psqi':sample_frequency,
    'dreams_psqi':sample_frequency,
    'pain_psqi':sample_frequency,
    'other_how_often_psqi':sample_frequency,
    'sleep_meds_psqi':sample_frequency,
    'stay_awake_psqi':sample_frequency,
    'snoring_psqi':sample_frequency,
    'pause_breath_psqi':sample_frequency,
    'twitching_psqi':sample_frequency,
    'confusion_psqi':sample_frequency,     
})

#date formatting
for i in range(samples.shape[0]):
    if samples['survey_date_psqi'][i] != '':
        tmp = samples['survey_date_psqi'][i]
        samples['survey_date_psqi'][i]=tmp.replace(tmp,tmp.split('/')[1].zfill(2)+'/'+tmp.split('/')[2].zfill(2)+'/'+tmp.split('/')[0])


#### Columns & Rows Deletion

In [182]:
# drop unuseful data entry
samples['patient_id'] = samples['patient_id'].str.upper()
samples_st = samples.loc[samples['patient_id'].str.contains('CFBR')].index.values[0]
samples_ed = samples.loc[samples['patient_id'].str.contains('CFBR')].index.values[-1]
samples = samples[samples_st: samples_ed+1].reset_index(drop=True)

# drop unuseful REDCap columns
samples.drop(['redcap_survey_identifier',
             'record_id',
              'timestamp_psqi',
             ], axis=1, inplace=True)

In [184]:
samples.columns

Index(['patient_id', 'survey_date_psqi', 'location_psqi',
       'usual_bed_time_psqi', 'number_of_mins_psqi', 'usual_get_up_psqi',
       'hours_of_sleep_psqi', 'sleep_in_30_mins_psqi', 'wake_in_night_psqi',
       'use_bathroom_psqi', 'breathe_comf_psqi', 'cough_snore_psqi',
       'cold_psqi', 'hot_psqi', 'dreams_psqi', 'pain_psqi',
       'other_reasons_psqi', 'other_how_often_psqi', 'quality_overall_psqi',
       'sleep_meds_psqi', 'stay_awake_psqi', 'enthusiasm_psqi', 'partner_psqi',
       'snoring_psqi', 'pause_breath_psqi', 'twitching_psqi', 'confusion_psqi',
       'other_restlessness_psqi', 'completion_psqi'],
      dtype='object')

In [185]:
# Calculate Score
#score 1: subjective_sleep_quality: quality_overall_psqi
samples["component1_score_psqi"]=quality_list

In [186]:
#score 2: sleep latency: num_mins_ls+in30_list
num_mins_ls=[]
result_ls=[]
def convert_mins_to_score(mins):
    if mins=='':
        score_mins=mins
    else:
        if int(mins) <= 15:
            score_mins=0
        elif int(mins) <= 30:
            score_mins=1
        elif int(mins) <= 60:
            score_mins=2
        elif int(mins) > 60:
            score_mins=3
    return(score_mins)

for mins in samples["number_of_mins_psqi"]:
    score=convert_mins_to_score(mins)
    num_mins_ls.append(score)

zipped_lists = zip(num_mins_ls, in30_list)

for i in zipped_lists:
    if i[0]=='':
        a=i[0]
    else:
        a=sum(i)
    result_ls.append(a)

samples["component2_score_psqi"]=result_ls

In [187]:
#score 3: hours_of_sleep_psqi
num_hours_ls=[]
def convert_hours_to_score(hours):
    if hours=='':
        score_hours=hours
    else:
        if int(hours) >= 7:
            score_hours=0
        elif int(hours) >= 6:
            score_hours=1
        elif int(hours) >= 5:
            score_hours=2
        elif int(hours) < 5:
            score_hours=3
    return(score_hours)

for hour in samples["hours_of_sleep_psqi"]:
    score=convert_hours_to_score(hour)
    num_hours_ls.append(score)
samples["component3_score_psqi"]=num_hours_ls

In [188]:
# score 4:
# calculate time in bed from 'usual_bed_time_psqi' and 'usual_get_up_psqi'
# get bed_time_ls and up_time_ls in 24h format
# problem : time denoted in 12 hours format, no am, pm information
# solution: for 'usual_bed_time_psqi' assum all hours >=7 and <12 to be in pm, else in am; 
# for 'usual_bed_time_psqi' assume assum all hours >=5 <=11 to be in am, else in pm

# problem2: error in self reported hours. For example, one person reported to be in bed for aorund 7 
# hours while the daily sleeping time is 8 hours.
# this would affect the calculation of sleep efficiency score, the person's sleep efficiency would exceed 100%. 

bed_time_ls, up_time_ls=[],[]
for item in samples['usual_bed_time_psqi']:
    if item == '':
        out_time=''
        bed_time_ls.append(out_time )
    else:
        if int(re.search('\d+',item).group(0)) >=7 and int(re.search('\d+',item).group(0)) < 12:
            bed_time=item + ' PM'
            out_time = datetime.strftime(datetime.strptime(bed_time, "%I:%M %p"), "%H:%M")
            bed_time_ls.append(out_time )
        else:
            bed_time=item + ' AM'
            out_time = datetime.strftime(datetime.strptime(bed_time, "%I:%M %p"), "%H:%M")
            bed_time_ls.append(out_time)
            
for item in samples['usual_get_up_psqi']:
    if item == '':
        out_time=''
        up_time_ls.append(out_time)
    else:
        if int(re.search('\d+',item).group(0)) >=5 and int(re.search('\d+',item).group(0)) <=11:
            bed_time=item + ' AM'
            out_time = datetime.strftime(datetime.strptime(bed_time, "%I:%M %p"), "%H:%M")
            up_time_ls.append(out_time )
        else:
            bed_time=item + ' PM'
            out_time = datetime.strftime(datetime.strptime(bed_time, "%I:%M %p"), "%H:%M")
            up_time_ls.append(out_time)

datetimeFormat = '%H:%M'
time1=bed_time_ls[0]
time2=up_time_ls[0]
diff = datetime.strptime(time2, datetimeFormat)- datetime.strptime(time1, datetimeFormat)
diff.seconds//3600

time_in_bed_ls=[]
datetimeFormat = '%H:%M'
for i in range(len(up_time_ls)):
    if up_time_ls[i]=='' or bed_time_ls[i]=='':
        a=''
    else:
        diff = datetime.strptime(up_time_ls[i], datetimeFormat)- datetime.strptime(bed_time_ls[i], datetimeFormat)
        a=diff.seconds//3600

    time_in_bed_ls.append(a)

# calculate sleep efficiency 
efficiency_ls=[]
for i in range(len(hours_list)):
    if hours_list[i]=='' or time_in_bed_ls[i]=='':
        eff=''
    else:
        eff=(hours_list[i]/time_in_bed_ls[i])*100
    efficiency_ls.append(eff)

# convert to score
score4_ls = []
def convert_to_score4(input):
    if input=='':
        score=input
    else:
        if int(input) > 85:
            score=0
        elif int(input) >= 75:
            score=1
        elif int(input) >=65:
            score=2
        else:
            score=3
    return(score)

for each in efficiency_ls:
    score4=convert_to_score4(each)
    score4_ls.append(score4)
samples["component4_score_psqi"]=score4_ls

In [189]:
#score 5
zipped_lists,result_ls,score5_ls=[],[],[]

zipped_lists = zip(wake_in_night_psqi_ls,use_bathroom_psqi_ls,breathe_comf_psqi_ls,cough_snore_psqi_ls,cold_psqi_ls,hot_psqi_ls,dreams_psqi_ls,pain_psqi_ls)
for i in zipped_lists:
    if i[0]=='':
        a=i[0]
    else:
        a=sum(i)
    result_ls.append(a)


def convert_to_score5(input):
    if input=='':
        score=input
    else:
        if int(input) == 0:
            score=0
        elif int(input) < 9:
            score=1
        elif int(input) <19:
            score=2
        else:
            score=3
    return(score)

for each in result_ls:
    score5=convert_to_score5(each)
    score5_ls.append(score5)
samples["component5_score_psqi"]=score5_ls

In [190]:
#score 6
samples["component6_score_psqi"]=sleep_meds_psqi_ls

In [191]:
#score 7
zipped_lists,result_ls,score7_ls=[],[],[]
zipped_lists = zip(stay_awake_psqi_ls, enthusiasm_psqi_ls)
for i in zipped_lists:
    if i[0]=='':
        a=i[0]
    else:
        a=sum(i)
    result_ls.append(a)
def convert_to_score7(input):
    if input=='':
        score=input
    else:
        if int(input) == 0:
            score=0
        elif int(input) < 2:
            score=1
        elif int(input) <4:
            score=2
        else:
            score=3
    return(score)
for each in result_ls:
    score7=convert_to_score7(each)
    score7_ls.append(score7)
samples["component7_score_psqi"]=score7_ls

In [192]:
#global PSQI score (add all 7 score together)
zipped_lists,global_score=[],[]
zipped_lists = zip(quality_list,result_ls,num_hours_ls,score4_ls,score5_ls,sleep_meds_psqi_ls,score7_ls)
for i in zipped_lists:
    if i[0]=='':
        a=i[0]
    else:
        a=sum(i)
    global_score.append(a)
samples["global_score_psqi"]=global_score

In [197]:
#find today's date
today = datetime.today().strftime('%m-%d-%Y')
#export
samples.to_csv (path+outfile+'_'+today+'.csv', index = False, header=True)
